In [1]:
import os
import sys
import yaml
import json
import numpy as np

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)

In [5]:
import ray
import modin
print(ray.__version__)
print(modin.__version__)

ray.shutdown()
ray.init(num_cpus = 4)
import modin.pandas as pd

0.7.3
0.6.1


2019-10-06 19:51:41,131	WARNING worker.py:1373 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-10-06 19:51:41,134	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-10-06_19-51-41_133872_3155/logs.
2019-10-06 19:51:41,245	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:40880 to respond...
2019-10-06 19:51:41,366	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:13055 to respond...
2019-10-06 19:51:41,371	INFO services.py:809 -- Starting Redis shard with 3.44 GB max memory.
2019-10-06 19:51:41,385	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-10-06_19-51-41_133872_3155/logs.
2019-10-06 19:51:41,389	WARNING services.py:1330 -- WARNING: The default object store size of 5.15 GB will use more than 50% of the available memory on this node (7.55 GB).

In [6]:
%%time

nyc_taxi_fare_file = os.path.join(data_path, cfg['data']['kaggle_nyc'])
nyc_taxi_fare_dtype = dict(zip(meta_data['kaggle_nyc']['colnames'], meta_data['kaggle_nyc']['dtype']))

#Note it doesn't like the parse datetime here. 
nyc_taxi_fare = pd.read_csv(nyc_taxi_fare_file, 
                            dtype = nyc_taxi_fare_dtype)

CPU times: user 3.68 s, sys: 1.41 s, total: 5.08 s
Wall time: 2min 6s
(pid=3405) 2019-10-06 19:55:53,940	INFO worker.py:426 -- The object with ID ObjectID(91a79a584cdc65d65e4aea8f58d3bc5702000000) already exists in the object store.


In [11]:
type(nyc_taxi_fare)

modin.pandas.dataframe.DataFrame

In [ ]:
%%time

nyc_taxi_fare.head()

In [ ]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].mean()

In [89]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].agg(count = len, 
                                                              mean = np.mean,
                                                              std = np.std,
                                                              min = np.min,
                                                              median = np.median)

CPU times: user 44.9 ms, sys: 4.94 ms, total: 49.9 ms
Wall time: 46.5 ms


,count,mean,std,min,median
passenger_count,,,,,
0,24.0,8.191667,4.095376,2.5,7.3
1,6973.0,11.405134,9.739049,2.5,8.5
2,1445.0,12.241952,13.548538,2.5,8.5
3,467.0,11.260214,10.085676,2.5,8.5
4,208.0,10.229519,8.600770,3.3,7.7
5,673.0,11.670223,9.499315,2.5,8.5
6,210.0,10.476476,7.071556,2.5,8.5


In [91]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].describe()

NameError: name 'nyc_taxi_fare' is not defined

In [90]:
del(nyc_taxi_fare)
ray.shutdown()

## MODIN INTERESTING BITS ##